---
title: "Trade Envy Report"
date: last-modified
engine: jupyter
execute:
  echo: false
  enabled: true
---

In [1]:
import pandas as pd
from cow_amm_trade_envy.datasources import get_logs
from cow_amm_trade_envy.models import Pools
import plotly.graph_objects as go


def analyze_surplus(file_path, pool_address):
    pool_address = pool_address.lower()

    def isin_pool(topic):
        assert "0x" == pool_address[:2]
        return pool_address[2:] in topic

    def log_is_used(log):
        return any(isin_pool(topic) for topic in log["topics"])

    def logs_are_used(logs):
        return any(log_is_used(log) for log in logs)

    df = pd.read_csv(file_path)

    df["logs"] = df["call_tx_hash"].apply(get_logs)
    df["is_used"] = df["logs"].apply(logs_are_used)

    df = df[df["trade_envy"] > 0]

    surplus_unused = df[~df["is_used"]]["trade_envy"].sum()
    surplus_used = df[df["is_used"]]["trade_envy"].sum()

    return {
        "total_unused_surplus": surplus_unused,
        "total_used_surplus": surplus_used,
    }

In [2]:
file_path = "data/cow_amm_missed_surplus.csv"
pool_address = Pools.USDC_WETH.ADDRESS

surplus_results = analyze_surplus(file_path, pool_address)

In [3]:
# Data for the bar plot
categories = ["Unused Potential", "Used Surplus"]
values = [
    surplus_results["total_unused_surplus"],
    surplus_results["total_used_surplus"],
]


fig = go.Figure(
    data=[
        go.Bar(
            name="Unused Surplus",
            x=[categories[0]],
            y=[values[0]],
            marker_color="green",
        ),
        go.Bar(
            name="Used Surplus", x=[categories[1]], y=[values[1]], marker_color="orange"
        ),
    ]
)

# Update layout for better visualization
fig.update_layout(
    title="Comparison of Surplus Usage",
    xaxis_title="Categories",
    yaxis_title="Surplus Amount",
    template="plotly",
    barmode="group",
)

# Show the plot
fig.show()

In [4]:
pool_address = pool_address.lower()


def isin_pool(topic):
    assert "0x" == pool_address[:2]
    return pool_address[2:] in topic


def log_is_used(log):
    return any(isin_pool(topic) for topic in log["topics"])


def logs_are_used(logs):
    return any(log_is_used(log) for log in logs)


df = pd.read_csv(file_path)

df["logs"] = df["call_tx_hash"].apply(get_logs)
df["is_used"] = df["logs"].apply(logs_are_used)